# Numerical Methods for Glacier Flow

## 1. Introduction to Glacier Flow
Glaciers are massive bodies of ice that flow slowly under their own weight. The flow of ice in a glacier is a complex process governed by physical laws that describe how the ice deforms and moves.
Given an initial glacier geometry, the time evolution in ice thickness $h(x, y, t)$ is determined by the mass conservation equation, which couples ice dynamics and surface mass balance (SMB)
through: 

$
\begin{aligned}
    \frac{\partial h}{\partial t} + \nabla \cdot (\mathbf{u}h) = SMB,
     \\
\end{aligned}
$

where $\nabla \cdot$ denotes the divergence operator with respect to the flux ($Q=\mathbf{u}h$). $\mathbf{u}$ is the vertically averaged horizontal ice velocity field and SMB the SMB function, which consists of the
integration of ice accumulation and ablation over one year. Mass conservation equation is generic and can be applied to model glacier evolution in number of applications provided adequate SMB and ice-flow model components. 
In the following, we mostly focus on developing an efficient numerical method to compute the ice-flow considering it is often the most computationally expensive component in glacier evolution model.

- First, we will use a numerical solution to compute $Q$. 
- Then we will use a data-driven Machine Learning (ML) approach that emulates ice-flow (Jouvet and others, 2022). 
The state of the art ML techniques ( Jouvet and Cordonnier, 2023), use a Physics Informed Neural Network (PINN); however, this goes beyond the scope of this chapter which focuses on hybrid models.






## 2. The Glen-Stokes Equations
The **Glen-Stokes model** is a widely used theoretical framework to describe this flow, particularly in the context of ice dynamics.

### 2.1 Stokes Equations for Glacier Flow
The Stokes equations are fundamental to describing the motion of viscous fluids, including glacier ice. For glaciers, the flow is generally assumed to be slow enough that inertial forces can be neglected, leaving us with the simplified Stokes equations:

$
\begin{aligned}
    -\nabla \cdot \sigma &= \rho \mathbf{g}, \\
    \nabla \cdot \mathbf{u} &= 0.
\end{aligned}
$

Where:
- $\sigma$ is the Cauchy stress tensor, which represents internal stresses within the glacier ice.
- $\rho$ is the density of ice (about 910 kg/m³).
- $\mathbf{g}$ is the gravitational acceleration vector.
- $\mathbf{u}$ is the velocity vector of the ice.

### 2.2 Glen's Flow Law
Glen's flow law relates the strain rate (deformation rate) of the ice to the applied stress. This relationship is crucial for understanding how ice deforms under pressure and is given by:

$
\mathbf{D}(\mathbf{u}) = \frac{1}{2} (\nabla \mathbf{u} + \nabla \mathbf{u}^\top)
$

The stress-strain relationship, often called Glen's law, is expressed as:

$
\mathbf{\tau} = 2\eta \mathbf{D}(\mathbf{u}),
$

Where:
- $\mathbf{\tau}$ is the deviatoric stress tensor (a measure of stress causing deformation).
- $\eta$ is the effective viscosity of ice, which depends on the temperature and the stress applied to the ice.
- $\mathbf{D}(\mathbf{u})$ is the strain rate tensor.

Glen's law is non-linear and can be expressed in the following form:

$
\mathbf{\tau} = 2 A^{-1/n} \left|\mathbf{D}(\mathbf{u})\right|^{\frac{1-n}{n}} \mathbf{D}(\mathbf{u}),
$

Where:
- $A$ is the flow rate factor, dependent on temperature.
- $n$ is Glen’s exponent, typically taken as 3 for glacier ice, indicating the non-linear relationship between stress and strain rate.
- $\left|\mathbf{D}(\mathbf{u})\right|$ is the magnitude of the strain rate tensor.



### 2.3. Transition to the Ice Sheet Equation

The Glen-Stokes equations describe the flow of ice under stress, but their computational cost makes them impractical for large-scale ice-sheet modeling over long time periods. This leads us to introduce the **shallow ice approximation (SIA)**, which is used in ice sheet models due to its simplicity and efficiency.

The SIA simplifies the full Stokes equations by assuming that the ice flow is dominated by vertical shear stresses and that horizontal stresses can be neglected. This results in an expression for the ice velocity in terms of the ice thickness gradient and the surface slope. The ice velocity $\mathbf{u}$ can be approximated as:

$
\mathbf{u} = -\left(\frac{2 A}{n+2}\right) \left(\rho g \sin(s)\right)^n h^{n+1} \nabla h,
$

where:
- $\rho$ is the ice density,
- $g$ is the gravitational acceleration,
- $A$ is Glen's flow rate factor,
- $n$ is Glen's law exponent (typically 3),
- $h$ is the ice thickness, and
- $s$ is the surface slope.

This equation governs the horizontal velocity of ice based on the local ice thickness and slope, and is more computationally feasible than the full Glen-Stokes model.
We plug the **u** into the mass conservation equation and obtain: 

$
\frac{\partial h}{\partial t} + \nabla \cdot (D(h,z)\frac{\partial z}{\partial x}) = \text{SMB},
$

where $D(h,z) =  f_d (\rho g)^3 h^5 |\nabla S|^2$.

This equation describes the time evolution of the ice thickness, where the velocity $\mathbf{u}$ is computed from the ice sheet's surface slope and thickness. This approach provides a balance between accuracy and computational efficiency and is widely used in large-scale ice sheet models.


## 3. Boundary Conditions

### 3.1 No-Slip Condition at the Base
In many glacier models, we assume a no-slip condition at the base, meaning the ice velocity is zero at the bedrock. This condition is suitable for glaciers frozen to their beds:

$
\mathbf{u} = 0 \text{ on the bedrock surface}.
$

### 3.2 Stress-Free Surface
At the glacier surface (exposed to air), a stress-free boundary condition is typically applied:

$
\sigma \cdot \mathbf{n} = 0 \text{ on the surface},
$

Where $\mathbf{n}$ is the outward normal vector at the glacier surface.

In [1]:
import numpy as np
import matplotlib.pyplot as plt  
from scipy import ndimage
import netCDF4
from IPython.display import clear_output
%matplotlib inline

In [2]:
# Physical parameters
Lx = 49700    # Domain length in x (m)
Ly = 32300    # Domain length in y (m)
ttot = 700   # Time limit (yr)
grad_b = 0.001 # Mass balance gradient (no unit)
b_max = 0.5   # Maximum precip (m/yr)
Z_ELA = 3000  # Elevation of equilibrium line altitude (m)
rho = 910.0   # Ice density (g/m^3)
g   = 9.81    # Earth's gravity (m/s^2)
fd  = 1e-18   # Deformation constant (Pa^-3 y^-1) 



# Initialization & load data

nout = 50  # Frequency of plotting
dtmax = 1   # maximum time step
dt = dtmax  # Initial time step
dx = 100
dy = 100  # Cell size in y
nx=int(Lx/dx)
ny=int(Ly/dy)
x = np.linspace(0, Lx, nx)  # x-coordinates
y = np.linspace(0, Ly, ny)  # y-coordinates

nc_file = netCDF4.Dataset('bedrock.nc')  # Load the NetCDF file
Z_topo = nc_file.variables['topg']    # Replace 'topg' with the appropriate v

H_ice = np.zeros((ny, nx))  # Initial ice thickness
Z_surf = Z_topo + H_ice  # Initial ice surface
time = 0  # Initial time
it = 0

In [ ]:

# Loop
while time < ttot:
    
    # Update time
    time += dt
    it   += 1


    # Calculate H_avg, size (ny-1,nx-1)
    H_avg = 0.25 * (H_ice[:-1, :-1] + H_ice[1:, 1:] + H_ice[:-1, 1:] + H_ice[1:, :-1])


    # Compute Snorm, size (ny-1,nx-1)
    Sx = np.diff(Z_surf, axis=1) / dx
    Sy = np.diff(Z_surf, axis=0) / dy
    Sx = 0.5 * (Sx[:-1, :] + Sx[1:, :])
    Sy = 0.5 * (Sy[:, :-1] + Sy[:, 1:])
    Snorm = np.sqrt(Sx**2 + Sy**2)


    # Compute D, size (ny-1,nx-1)
    D = fd * (rho * g)**3.0 * H_avg**5 * Snorm**2


    # Compute dt
    dt = min(min(dx, dy)**2 / (4.1 * np.max(D)), dtmax)


    # Compute qx, size (ny-2,nx-1)
    qx = -(0.5 * (D[:-1,:] + D[1:,:])) * np.diff(Z_surf[1:-1,:], axis=1) / dx


    # Compute qy, size (ny-1,nx-2)
    qy = -(0.5 * (D[:,:-1] + D[:,1:])) * np.diff(Z_surf[:,1:-1,], axis=0) / dy


    # Update rule (diffusion)
    dHdt = -(np.diff(qx, axis=1) / dx + np.diff(qy, axis=0) / dy)
    H_ice[1:-1, 1:-1] += dt * dHdt # size (ny-2,nx-2)


    b = np.minimum(grad_b * (Z_surf - Z_ELA), b_max)


    # Update rule (mass balance)
    H_ice[1:-1, 1:-1] += dt * b[1:-1, 1:-1]


    # Update rule (positive thickness)
    H_ice = np.maximum(H_ice, 0)


    # updatesurface topography
    Z_surf = Z_topo + H_ice
  
    # Update ELA after 500 years
    if time > 500:
        Z_ELA = 2700

    # Display
    if it % nout == 0:
        clear_output(wait=True)  # Clear the previous output in the notebook
    
        plt.figure(2, figsize=(11, 4), dpi=200)
        
        # First subplot: Ice surface
        plt.subplot(1, 2, 1)
        plt.imshow(Z_surf, extent=[0, Lx/1000, 0, Ly/1000], cmap='terrain', origin='lower')
        plt.colorbar(label='Elevation (m)')
        plt.title('Ice Surface at ' + str(int(time)) + ' y')
        plt.xlabel('Distance, km')
        plt.ylabel('Distance, km')
    
        # Second subplot: Ice thickness
        plt.subplot(1, 2, 2)
        plt.imshow(np.where(H_ice > 0, H_ice, np.nan), extent=[0, Lx/1000, 0, Ly/1000], cmap='jet', origin='lower')
        plt.colorbar(label='Ice Thickness (m)')
        plt.title('Ice Thickness at ' + str(int(time)) + ' y')
        plt.xlabel('Distance, km')
        plt.ylabel('Distance, km')
        
        # Show the plot
        plt.show()
    


## 4. Emulating Ice Flow with Machine Learning

The *Instructed Glacier Model* (IGM) presented by Jouvet et al. introduces a convolutional neural network (CNN) to predict ice flow, trained using data from traditional models such as hybrid SIA+SSA or Stokes models. The advantage of this approach is that it substitutes the computationally expensive ice flow component with a much faster emulator. This enables simulations that are up to 1000 times faster, with a fidelity of over 90%.

### 4.1 Overview of the Machine Learning Approach

Machine learning, specifically deep learning, allows us to create surrogate models that emulate the glacier dynamics described by traditional models. These surrogate models, once trained, provide nearly identical ice flow solutions at a fraction of the computational cost.

- **CNN Emulator**: A convolutional neural network (CNN) is trained on input data such as ice thickness, surface slope, and basal sliding coefficients, and it predicts the ice flow field (horizontal velocity).
  
- **Training**: The network is trained using data generated from physically accurate glacier models like PISM (Parallel Ice Sheet Model) or CfsFlow. This large dataset is crucial for high fidelity and ensuring the emulator can generalize across different glaciers.

In [ ]:
# I suggest to use Tensor Flow variables instead of Numpy
import xarray as xr
import tensorflow as tf
from tensorflow.keras.models import load_model

train_mode=False

In [5]:
# Function to load, merge NetCDF files, and fill NaN with 0
def load_and_merge_netcdf(files):
    datasets = [xr.open_dataset(file) for file in files]
    
    # Align the datasets along the spatial dimensions using fill_value=0 for NaN
    combined_ds = xr.concat(datasets, dim='time', join='outer').fillna(0)
    
    return combined_ds

# Function to scale the field based on 90th percentile of its maximum
def scale_field(field):
    max_90th_percentile = np.percentile(field.max(axis=(1, 2)), 90)
    return field / max_90th_percentile

# Function to augment the data by flipping and adding noise
def augment_data(inputs, outputs):
    aug_inputs, aug_outputs = [], []

    for inp, out in zip(inputs, outputs):
        # Original data
        aug_inputs.append(inp)
        aug_outputs.append(out)

        # Horizontal flip
        aug_inputs.append(np.flip(inp, axis=2))  # Flip along x-axis
        aug_outputs.append(np.flip(out, axis=2))

        # Vertical flip
        aug_inputs.append(np.flip(inp, axis=1))  # Flip along y-axis
        aug_outputs.append(np.flip(out, axis=1))

        # Add random Gaussian noise
        noise_factor = 0.05
        aug_inputs.append(inp + noise_factor * np.random.normal(size=inp.shape))
        aug_outputs.append(out + noise_factor * np.random.normal(size=out.shape))

    return np.array(aug_inputs), np.array(aug_outputs)

import matplotlib.pyplot as plt

# Plot input and output fields side by side with color bars
def plot_input_output(thk, slopesurx, slopesury, ubar, vbar):
    fig, axs = plt.subplots(2, 3, figsize=(18, 12))

    # Plot inputs
    im0 = axs[0, 0].imshow(thk, cmap='Blues')
    axs[0, 0].set_title('Ice Thickness (thk)')
    axs[0, 0].set_xlabel('x')
    axs[0, 0].set_ylabel('y')
    fig.colorbar(im0, ax=axs[0, 0], orientation='vertical')

    im1 = axs[0, 1].imshow(slopesurx, cmap='RdBu')
    axs[0, 1].set_title('Surface Slope x (slopesurx)')
    axs[0, 1].set_xlabel('x')
    axs[0, 1].set_ylabel('y')
    fig.colorbar(im1, ax=axs[0, 1], orientation='vertical')

    im2 = axs[0, 2].imshow(slopesury, cmap='RdBu')
    axs[0, 2].set_title('Surface Slope y (slopesury)')
    axs[0, 2].set_xlabel('x')
    axs[0, 2].set_ylabel('y')
    fig.colorbar(im2, ax=axs[0, 2], orientation='vertical')

    # Plot outputs
    im3 = axs[1, 0].imshow(ubar, cmap='viridis')
    axs[1, 0].set_title('Velocity x (ubar)')
    axs[1, 0].set_xlabel('x')
    axs[1, 0].set_ylabel('y')
    fig.colorbar(im3, ax=axs[1, 0], orientation='vertical')

    im4 = axs[1, 1].imshow(vbar, cmap='viridis')
    axs[1, 1].set_title('Velocity y (vbar)')
    axs[1, 1].set_xlabel('x')
    axs[1, 1].set_ylabel('y')
    fig.colorbar(im4, ax=axs[1, 1], orientation='vertical')

    # Adjust layout
    plt.tight_layout()
    plt.show()



In [6]:
# File paths (Replace with your actual file paths)
file_paths = ['/home/klleshi/Desktop/surflib3d_A78_100/ALP11_A78_C0/ex.nc']

# Load and merge the NetCDF datasets
merged_data = load_and_merge_netcdf(file_paths)

In [ ]:
# Step 1: Extract the input and output fields
thk = merged_data['thk'].values           # Ice thickness
slopsurfx = merged_data['slopsurfx'].values # Surface slope in x
slopsurfy = merged_data['slopsurfy'].values # Surface slope in y
ubar = merged_data['ubar'].values         # Velocity x component
vbar = merged_data['vbar'].values         # Velocity y component
usurf = merged_data['usurf'].values

# Calculate 90th percentile scaling factors for training
scaling_factors = {
    "thk": np.percentile(thk.max(axis=(1, 2)), 90),
    "slopsurfx": np.percentile(slopsurfx.max(axis=(1, 2)), 90),
    "slopsurfy": np.percentile(slopsurfy.max(axis=(1, 2)), 90),
    "ubar": np.percentile(ubar.max(axis=(1, 2)), 90),
    "vbar": np.percentile(vbar.max(axis=(1, 2)), 90)
}

# Step 2: Scale each field using the 90th percentile of its maximum
thk_scaled = scale_field(thk)
slopsurfx_scaled = scale_field(slopsurfx)
slopsurfy_scaled = scale_field(slopsurfy)
ubar_scaled = scale_field(ubar)
vbar_scaled = scale_field(vbar)
usurf_scaled = scale_field(usurf)

# Step 3: Stack inputs and outputs after scaling
inputs_scaled = np.stack([thk_scaled, slopsurfx_scaled, slopsurfy_scaled], axis=-1)  # Shape: (time, y, x, 3)
outputs_scaled = np.stack([ubar_scaled, vbar_scaled], axis=-1)  # Shape: (time, y, x, 2)

# Check shapes
print(f"Inputs scaled shape: {inputs_scaled.shape}")
print(f"Outputs scaled shape: {outputs_scaled.shape}")

In [ ]:
slopsurfy_scaled.shape

In [ ]:
# Plotting the data to visualize the relationship between input and output
time_idx=20
plot_input_output(thk[time_idx], slopsurfx[time_idx], slopsurfy[time_idx], ubar[time_idx], vbar[time_idx])

#### Start working on the CNN model

In [8]:
# Configuration as a dictionary
config = {
    "nb_layers": 4,               # Number of convolutional layers
    "nb_out_filter": 32,           # Number of output filters for Conv2D
    "conv_ker_size": 3,            # Convolution kernel size
    "activation": "relu",          # Activation function: "relu" or "lrelu"
    "dropout_rate": 0.1,           # Dropout rate
    "regularization": 0.0001       # L2 regularization
}


In [9]:
def build_cnn(nb_inputs, nb_outputs, config):
    """
    Build a convolutional neural network (CNN) for glacier velocity field prediction.
    
    Parameters:
    - nb_inputs: Number of input channels (thk, slopsurfx, slopsurfy).
    - nb_outputs: Number of output channels (ubar, vbar).
    - config: Dictionary containing CNN configuration.
    
    Returns:
    - A compiled Keras model.
    """
    
    # Define the input layer
    inputs = tf.keras.layers.Input(shape=[None, None, nb_inputs])

    conv = inputs

    # Activation function choice
    if config['activation'] == "lrelu":
        activation = tf.keras.layers.LeakyReLU(alpha=0.01)
    else:
        activation = tf.keras.layers.ReLU()

    # Stack convolutional layers
    for i in range(config['nb_layers']):
        conv = tf.keras.layers.Conv2D(
            filters=config['nb_out_filter'],
            kernel_size=(config['conv_ker_size'], config['conv_ker_size']),
            kernel_regularizer=tf.keras.regularizers.l2(config['regularization']),
            padding="same"
        )(conv)

        conv = activation(conv)

        conv = tf.keras.layers.Dropout(config['dropout_rate'])(conv)

    # Output layer with nb_outputs channels (for ubar, vbar)
    outputs = tf.keras.layers.Conv2D(
        filters=nb_outputs, 
        kernel_size=(1, 1), 
        activation=None
    )(conv)

    # Return the complete model
    model = tf.keras.models.Model(inputs=inputs, outputs=outputs)
    
    return model

In [ ]:
# Build and Compile the Model

# Define the number of input channels (thk, slopsurfx, slopsurfy) and output channels (ubar, vbar)
nb_inputs = 3  # thk, slopsurfx, slopsurfy
nb_outputs = 2  # ubar, vbar

# Build the CNN model
model = build_cnn(nb_inputs, nb_outputs, config)

# Compile the model
model.compile(optimizer='adam', loss='mae', metrics=["mae", "mse"])

# Print model summary
model.summary()


In [ ]:
# Split index for 90-10 split
split_idx = int(0.9 * inputs_scaled.shape[0])

# Train-test split for inputs
X_train = inputs_scaled[:split_idx, :, :, :]
X_test = inputs_scaled[split_idx:, :, :, :]

# Train-test split for outputs
y_train = outputs_scaled[:split_idx, :, :, :]
y_test = outputs_scaled[split_idx:, :, :, :]

# Apply data augmentation
X_train_aug, y_train_aug = augment_data(X_train, y_train)
# Check shapes
print(f"X_train shape: {X_train_aug.shape}")
print(f"y_train shape: {y_train_aug.shape}")
print(f"X_test shape: {X_test.shape}")
print(f"y_test shape: {y_test.shape}")


In [12]:
if train_mode: 
# Train the model
    history = model.fit(X_train, y_train, batch_size=8, epochs=70, validation_data=(X_test, y_test))
    # Save the trained model to a file
    model.save('glacier_flow_model.h5')
else:
    model = load_model('glacier_flow_model.h5')

In [ ]:
# Step 3: Plot the learning curves for training and validation loss
plt.figure(figsize=(10, 6))

# Plot training loss
plt.plot(history.history['loss'], label='Training Loss')

# Plot validation loss
plt.plot(history.history['val_loss'], label='Validation Loss')

# Add labels and legend
plt.title('Learning Curve (Loss vs. Epochs)')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend(loc='upper right')
plt.savefig("Lr.png")
# Show plot
plt.show()


In [ ]:
# Function to plot inputs, true outputs, and predicted outputs
def plot_comparison(thk, slopesurx, slopesury, true_ubar, true_vbar, pred_ubar, pred_vbar, time_idx=0):
    fig, axs = plt.subplots(3, 3, figsize=(18, 18))

    # Plot inputs
    axs[0, 0].imshow(thk[time_idx, :, :], cmap='Blues')
    axs[0, 0].set_title('Ice Thickness (thk)')
    axs[0, 0].set_xlabel('x')
    axs[0, 0].set_ylabel('y')

    axs[0, 1].imshow(slopesurx[time_idx, :, :], cmap='RdBu')
    axs[0, 1].set_title('Surface Slope x (slopesurx)')
    axs[0, 1].set_xlabel('x')
    axs[0, 1].set_ylabel('y')

    axs[0, 2].imshow(slopesury[time_idx, :, :], cmap='RdBu')
    axs[0, 2].set_title('Surface Slope y (slopesury)')
    axs[0, 2].set_xlabel('x')
    axs[0, 2].set_ylabel('y')

    # Plot true outputs
    axs[1, 0].imshow(true_ubar[time_idx, :, :], cmap='viridis')
    axs[1, 0].set_title('True Velocity x (ubar)')
    axs[1, 0].set_xlabel('x')
    axs[1, 0].set_ylabel('y')

    axs[1, 1].imshow(true_vbar[time_idx, :, :], cmap='viridis')
    axs[1, 1].set_title('True Velocity y (vbar)')
    axs[1, 1].set_xlabel('x')
    axs[1, 1].set_ylabel('y')

    # Plot predicted outputs
    axs[2, 0].imshow(pred_ubar[time_idx, :, :], cmap='viridis')
    axs[2, 0].set_title('Predicted Velocity x (ubar)')
    axs[2, 0].set_xlabel('x')
    axs[2, 0].set_ylabel('y')

    axs[2, 1].imshow(pred_vbar[time_idx, :, :], cmap='viridis')
    axs[2, 1].set_title('Predicted Velocity y (vbar)')
    axs[2, 1].set_xlabel('x')
    axs[2, 1].set_ylabel('y')

    # Adjust layout
    plt.tight_layout()
    plt.show()

# Assuming you have a trained model and test data loaded, let's make predictions
predicted_outputs = model.predict(X_test)

# Separate predicted outputs into ubar and vbar components
pred_ubar = predicted_outputs[..., 0]  # First channel is ubar
pred_vbar = predicted_outputs[..., 1]  # Second channel is vbar

# Call the plot function for a specific time index, e.g., 0
time_idx = 0
plot_comparison(X_test[..., 0], X_test[..., 1], X_test[..., 2], y_test[..., 0], y_test[..., 1], pred_ubar, pred_vbar, time_idx)


### 4.2 Implementation

A basic version of this model could use the following steps:
1. **Input Variables**: The CNN takes ice thickness and surface slope gradients as inputs  $\{ h(x,y), \frac{\partial s}{\partial x},\frac{\partial s}{\partial y}\}$.
2. **Training the Emulator**: The model is trained using a dataset generated from high-order glacier flow models.
3. **Prediction**: Once trained, the emulator predicts the vertically averaged ice flow from the input variables **u**(u,v).
    
 The input and output fields are 2 D grid rasters: 

$
\R^{N_x \times N_y \times 3} \rightarrow \R^{N_x \times N_y \times 2} ,
$


For the hybrid model we will calculate **u**(x,y) with the emulator we trained above and plug it in the mass conservation equation to calculate the flux. 
The implementation is similar to above with some slight changes. Now we will need a `compute_divflux()` function for each itteration of the numerical scheme.
In addition, we will need a new function (`compute_gradient_tf()`) to calculate the slope of the glacier, remember that the velocity field is mainly dependent on the slope and the thckness of the glacier.



In [21]:
@tf.function()
def compute_divflux(u, v, h, dx, dy):
    """
    Upwind computation of the divergence of the flux: d(u h)/dx + d(v h)/dy
    
    Parameters:
    - u: x-component of velocity (2D tensor).
    - v: y-component of velocity (2D tensor).
    - h: ice thickness (2D tensor).
    - dx: grid spacing in the x-direction (float).
    - dy: grid spacing in the y-direction (float).
    
    Returns:
    - divflux: divergence of flux (2D tensor).
    """
    # Compute u and v on the staggered grid
    u = tf.concat([u[:, 0:1], 0.5 * (u[:, :-1] + u[:, 1:]), u[:, -1:]], 1)  # shape (ny, nx+1)
    v = tf.concat([v[0:1, :], 0.5 * (v[:-1, :] + v[1:, :]), v[-1:, :]], 0)  # shape (ny+1, nx)

    # Extend h with constant value at the domain boundaries
    Hx = tf.pad(h, [[0, 0], [1, 1]], "CONSTANT")  # shape (ny, nx+2)
    Hy = tf.pad(h, [[1, 1], [0, 0]], "CONSTANT")  # shape (ny+2, nx)

    # Compute fluxes by selecting the upwind quantities
    Qx = u * tf.where(u > 0, Hx[:, :-1], Hx[:, 1:])  # shape (ny, nx+1)
    Qy = v * tf.where(v > 0, Hy[:-1, :], Hy[1:, :])  # shape (ny+1, nx)

    # Compute the divergence, final shape is (ny, nx)
    divflux = (Qx[:, 1:] - Qx[:, :-1]) / dx + (Qy[1:, :] - Qy[:-1, :]) / dy
    return divflux


@tf.function()
def compute_gradient_tf(s, dx, dy):
    """
    Compute spatial 2D gradient of a given field.
    
    Parameters:
    - s: surface elevation (2D tensor).
    - dx: grid spacing in the x-direction (float).
    - dy: grid spacing in the y-direction (float).
    
    Returns:
    - diffx: gradient in the x-direction (2D tensor).
    - diffy: gradient in the y-direction (2D tensor).
    """
    EX = tf.concat([1.5 * s[:, 0:1] - 0.5 * s[:, 1:2], 0.5 * s[:, :-1] + 0.5 * s[:, 1:], 1.5 * s[:, -1:] - 0.5 * s[:, -2:-1]], 1)
    diffx = (EX[:, 1:] - EX[:, :-1]) / dx

    EY = tf.concat([1.5 * s[0:1, :] - 0.5 * s[1:2, :], 0.5 * s[:-1, :] + 0.5 * s[1:, :], 1.5 * s[-1:, :] - 0.5 * s[-2:-1, :]], 0)
    diffy = (EY[1:, :] - EY[:-1, :]) / dy

    return diffx, diffy


def apply_boundary_condition(H_ice, boundary_width=5):
    """
    Apply boundary condition to the ice thickness field `H_ice`.
    The ice thickness will linearly decrease to zero starting from `boundary_width` pixels away from the boundary.
    
    Parameters:
    - H_ice: 2D numpy array representing ice thickness.
    - boundary_width: Number of pixels from the boundary where H_ice starts to decrease.
    
    Returns:
    - Modified H_ice with boundary condition applied.
    """
    ny, nx = H_ice.shape  # Get the dimensions of the ice thickness field

    # Create linear ramps
    ramp = np.linspace(1, 0, boundary_width)  # Ramp that linearly decreases from 1 to 0

    # Apply boundary condition to the left boundary
    H_ice[:, :boundary_width] *= ramp[::-1]  # Decrease from boundary to 5 pixels inwards

    # Apply boundary condition to the right boundary
    H_ice[:, -boundary_width:] *= ramp  # Decrease from 5 pixels inwards to the boundary

    # Apply boundary condition to the top boundary
    H_ice[:boundary_width, :] *= ramp[::-1, np.newaxis]  # Decrease vertically from top boundary

    # Apply boundary condition to the bottom boundary
    H_ice[-boundary_width:, :] *= ramp[:, np.newaxis]  # Decrease vertically to bottom boundary
    
    return H_ice


In [27]:
# Physical parameters
Lx = 49700    # Domain length in x (m)
Ly = 32300    # Domain length in y (m)
ttot = 700   # Time limit (yr)
grad_b = 0.001 # Mass balance gradient (no unit)
b_max = 0.5   # Maximum precip (m/yr)
Z_ELA = 3000  # Elevation of equilibrium line altitude (m)




# Initialization & load data

nout = 50  # Frequency of plotting
dtmax = 1   # maximum time step
cfl = 0.20
dx = 100
dy = 100  # Cell size in y
nx=int(Lx/dx)
ny=int(Ly/dy)
x = np.linspace(0, Lx, nx)  # x-coordinates
y = np.linspace(0, Ly, ny)  # y-coordinates

nc_file = netCDF4.Dataset('bedrock.nc')  # Load the NetCDF file
Z_topo = nc_file.variables['topg']  # Replace 'topg' with the appropriate v



H_ice = np.zeros((ny, nx))  # Initial ice thickness
Z_surf = Z_topo + H_ice  # Initial ice surface
# Compute gradients of surface elevation (slopes)
slopsurfx, slopsurfy = compute_gradient_tf(Z_surf, dx, dx)
time = tf.cast(0.0, tf.float32)  # Initial time as float32
dt = tf.cast(dtmax, tf.float32)  # Cast dtmax to float32e
it = 0

In [ ]:


# Ensure all arrays are float32
H_ice = tf.cast(H_ice, tf.float32)
slopsurfx = tf.cast(slopsurfx, tf.float32)
slopsurfy = tf.cast(slopsurfy, tf.float32)
Z_surf = tf.cast(Z_surf, tf.float32)
Z_topo = tf.cast(Z_topo, tf.float32)

#Fields need to be scaled

# Loop
while time < ttot:
    # Update time
    time += dt
    it += 1

    # Calculate H_avg, size (ny-1, nx-1)
    H_avg = 0.25 * (H_ice[:-1, :-1] + H_ice[1:, 1:] + H_ice[:-1, 1:] + H_ice[1:, :-1])
    
    # Scale the inputs with stored scaling factors
    H_ice_scaled = H_ice / scaling_factors["thk"]
    slopsurfx_scaled = slopsurfx / scaling_factors["slopsurfx"]
    slopsurfy_scaled = slopsurfy / scaling_factors["slopsurfy"]

    # Combine scaled inputs
    input_data_scaled = np.stack([H_ice_scaled, slopsurfx_scaled, slopsurfy_scaled], axis=-1)
    input_data_scaled = np.expand_dims(input_data_scaled, axis=0)  # Add batch dimension
    # Step 2: Use the trained model to predict ubar (x-velocity) and vbar (y-velocity)
    
    ubar_vbar_pred = model.predict(input_data_scaled, verbose=0)
    ubar = ubar_vbar_pred[0, :, :, 0] * scaling_factors["ubar"] # x-component of velocity (ubar)
    vbar = ubar_vbar_pred[0, :, :, 1] * scaling_factors["vbar"] # y-component of velocity (vbar)

    # Step 3: Compute maximum velocity for CFL condition
    vel_max = max(
        tf.math.reduce_max(tf.math.abs(ubar)),
        tf.math.reduce_max(tf.math.abs(vbar)),
    ).numpy()
    
    # Step 4: Compute time step (CFL condition)
    dt = tf.cast(tf.minimum(cfl * dx / vel_max, dtmax), tf.float32)
  
    # Step 5: Update rule (diffusion): Compute the change in thickness (dH/dt)
    dHdt = -compute_divflux(ubar, vbar, H_ice, dx, dx)
    
    # Update ice thickness (ensure no negative values)
    H_ice += dt * dHdt

    # Define the SMB (Surface Mass Balance)
    b = tf.minimum(grad_b * (Z_surf - Z_ELA), b_max)

    # Update rule (mass balance)
    H_ice += dt * b

    # Update rule (positive thickness)
    H_ice = np.maximum(H_ice, 0)
    
    # Apply the boundary condition before the next iteration
    H_ice = apply_boundary_condition(H_ice)

    # Update surface topography
    Z_surf = Z_topo + H_ice

    # Compute gradients of surface elevation (slopes)
    slopsurfx, slopsurfy = compute_gradient_tf(Z_surf, dx, dx)

    # Update ELA after 500 years
    if time > 500:
        Z_ELA = 2700
    
   # Display
    if it % nout == 0:
        clear_output(wait=True)  # Clear the previous output in the notebook

        plt.figure(2, figsize=(11, 4), dpi=200)

        # First subplot: Ice surface
        plt.subplot(1, 2, 1)
        plt.imshow(Z_surf, extent=[0, Lx/1000, 0, Ly/1000], cmap='terrain', origin='lower')
        plt.colorbar(label='Elevation (m)')
        plt.title('Ice Surface at ' + str(int(time)) + ' y')
        plt.xlabel('Distance, km')
        plt.ylabel('Distance, km')

        # Second subplot: Ice thickness
        plt.subplot(1, 2, 2)
        plt.imshow(np.where(H_ice > 0, H_ice, np.nan), extent=[0, Lx/1000, 0, Ly/1000], cmap='jet', origin='lower')
        plt.colorbar(label='Ice Thickness (m)')
        plt.title('Ice Thickness at ' + str(int(time)) + ' y')
        plt.xlabel('Distance, km')
        plt.ylabel('Distance, km')

        # Show the plot
        plt.show()